In [1]:
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from collections import deque
import os
import hashlib
import json
import pickle
import csv

# Configuration for 4x4 grid
GRID_SIZE = 4
EXIT_POS = (1, GRID_SIZE - 1)  # Exit at row 1 (0-indexed), rightmost column

class RushHour4x4:
    def __init__(self):
        self.grid_size = GRID_SIZE
        self.exit_pos = EXIT_POS

    def get_difficulty_config_for_base(self, base_puzzle_num):
        """
        Get difficulty configuration based on base puzzle number (1-150).
        Ensures exactly 50 easy, 50 medium, 50 hard base puzzles.
        """
        # Define difficulty levels with (num_1x1, num_2x1, difficulty_name)
        difficulty_configs = {
            "easy": [(4, 0, "easy"), (3, 1, "easy"), (5, 0, "easy")],
            "medium": [(2, 2, "medium"), (3, 2, "medium"), (1, 2, "medium"), (4, 1, "medium")],
            "hard": [(0, 3, "hard"), (1, 3, "hard"), (2, 3, "hard"), (0, 4, "hard")]
        }
        
        # Strict distribution: puzzles 1-50 easy, 51-100 medium, 101-150 hard
        if base_puzzle_num <= 50:
            return random.choice(difficulty_configs["easy"])
        elif base_puzzle_num <= 100:
            return random.choice(difficulty_configs["medium"])
        else:
            return random.choice(difficulty_configs["hard"])

    def create_empty_grid(self):
        """Create empty 4x4 grid"""
        return [['.' for _ in range(self.grid_size)] for _ in range(self.grid_size)]
    
    def find_piece_positions(self, grid, piece_id):
        """Find all positions occupied by a piece"""
        positions = []
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == piece_id:
                    positions.append((r, c))
        return positions
    
    def can_place_piece(self, grid, positions):
        """Check if a piece can be placed at given positions"""
        for r, c in positions:
            if not (0 <= r < self.grid_size and 0 <= c < self.grid_size):
                return False
            if grid[r][c] != '.':
                return False
        return True
    
    def place_piece(self, grid, piece_id, positions):
        """Place a piece on the grid"""
        for r, c in positions:
            grid[r][c] = piece_id
    
    def remove_piece(self, grid, piece_id):
        """Remove a piece from the grid"""
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == piece_id:
                    grid[r][c] = '.'
    
    def get_all_empty_positions(self, grid):
        """Get all empty positions in the grid"""
        empty = []
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] == '.':
                    empty.append((r, c))
        return empty
    
    def generate_2x1_positions(self, start_pos, orientation):
        """Generate positions for a 2x1 blocker"""
        r, c = start_pos
        if orientation == 'horizontal':
            return [(r, c), (r, c + 1)]
        else:  # vertical
            return [(r, c), (r + 1, c)]
    
    def generate_puzzle(self, num_1x1_blockers=3, num_2x1_blockers=2, max_attempts=1000):
        """Generate a random 4x4 Rush Hour puzzle"""
        for _ in range(max_attempts):
            grid = self.create_empty_grid()
            
            # Place car (1x1) - not at exit position
            available_positions = [(r, c) for r in range(self.grid_size) 
                                   for c in range(self.grid_size) if (r, c) != self.exit_pos]
            car_pos = random.choice(available_positions)
            self.place_piece(grid, 'C', [car_pos])
            
            # Place 2x1 blockers first
            blockers_placed = 0
            for i in range(num_2x1_blockers):
                piece_id = f'H{i+1}'  # H for 2x1 pieces
                placed = False
                attempts_for_this_piece = 0
                
                while not placed and attempts_for_this_piece < 50:
                    orientation = random.choice(['horizontal', 'vertical'])
                    if orientation == 'horizontal':
                        valid_starts = [(r, c) for r in range(self.grid_size) 
                                        for c in range(self.grid_size - 1)]
                    else:  # vertical
                        valid_starts = [(r, c) for r in range(self.grid_size - 1) 
                                        for c in range(self.grid_size)]
                    
                    if valid_starts:
                        start_pos = random.choice(valid_starts)
                        positions = self.generate_2x1_positions(start_pos, orientation)
                        if self.can_place_piece(grid, positions):
                            self.place_piece(grid, piece_id, positions)
                            placed = True
                            blockers_placed += 1
                    attempts_for_this_piece += 1
            
            # Place 1x1 blockers
            for i in range(num_1x1_blockers):
                piece_id = f'B{i+1}'
                empty_positions = self.get_all_empty_positions(grid)
                if empty_positions:
                    pos = random.choice(empty_positions)
                    self.place_piece(grid, piece_id, [pos])
                    blockers_placed += 1
            
            # Check if we have enough blockers and the puzzle is solvable
            expected_total = num_2x1_blockers + num_1x1_blockers
            if blockers_placed >= max(1, expected_total - 1):  # Allow some flexibility
                solution = self.bfs_solve(grid)
                if solution is not None and len(solution) > 0:  # At least 1 move
                    return grid
        
        raise Exception(f"Could not generate solvable puzzle after {max_attempts} attempts")
    
    def grid_to_tuple(self, grid):
        """Convert grid to hashable tuple"""
        return tuple(tuple(row) for row in grid)
    
    def tuple_to_grid(self, t):
        """Convert tuple back to grid"""
        return [list(row) for row in t]
    
    def is_solved(self, grid):
        """Check if car 'C' is at exit position"""
        return grid[self.exit_pos[0]][self.exit_pos[1]] == 'C'
    
    def get_piece_info(self, grid, piece_id):
        """Get information about a piece (positions and type)"""
        positions = self.find_piece_positions(grid, piece_id)
        if len(positions) == 1:
            return positions, '1x1'
        elif len(positions) == 2:
            r1, c1 = positions[0]
            r2, c2 = positions[1]
            if r1 == r2:
                return sorted(positions, key=lambda x: x[1]), 'horizontal'
            else:
                return sorted(positions, key=lambda x: x[0]), 'vertical'
        return positions, 'unknown'
    
    def get_neighbors(self, state):
        """Get all valid neighboring states (move any piece by 1 in any direction)"""
        grid = self.tuple_to_grid(state)
        neighbors = []
        
        # Find all unique pieces
        pieces = set()
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                if grid[r][c] != '.':
                    pieces.add(grid[r][c])
        
        # Try moving each piece in all 4 directions
        for piece_id in pieces:
            positions, _ = self.get_piece_info(grid, piece_id)
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:  # up, down, left, right
                new_positions = [(r + dr, c + dc) for r, c in positions]
                
                # Bounds check
                valid_move = True
                for nr, nc in new_positions:
                    if not (0 <= nr < self.grid_size and 0 <= nc < self.grid_size):
                        valid_move = False
                        break
                if not valid_move:
                    continue
                
                # Collision check
                temp_grid = [row[:] for row in grid]
                self.remove_piece(temp_grid, piece_id)
                can_move = True
                for nr, nc in new_positions:
                    if temp_grid[nr][nc] != '.':
                        can_move = False
                        break
                if not can_move:
                    continue
                
                # Apply move
                new_grid = [row[:] for row in temp_grid]
                self.place_piece(new_grid, piece_id, new_positions)
                neighbors.append(self.grid_to_tuple(new_grid))
        
        return neighbors
    
    def bfs_solve(self, start_grid):
        """BFS solver that returns ONE optimal solution"""
        start_state = self.grid_to_tuple(start_grid)
        if self.is_solved(start_grid):
            return []  # Already solved
        
        queue = deque([(start_state, [])])
        visited = {start_state}
        
        while queue:
            current_state, path = queue.popleft()
            
            for next_state in self.get_neighbors(current_state):
                if next_state in visited:
                    continue
                    
                visited.add(next_state)
                new_path = path + [next_state]
                
                if self.is_solved(self.tuple_to_grid(next_state)):
                    return new_path
                
                queue.append((next_state, new_path))
        
        return None
    
    def verify_solution(self, start_grid, solution_path):
        """Verify that a solution path is valid"""
        if not solution_path:
            return self.is_solved(start_grid)
        
        current = self.grid_to_tuple(start_grid)
        for i, next_state in enumerate(solution_path):
            valid_neighbors = self.get_neighbors(current)
            if next_state not in valid_neighbors:
                print(f"Invalid move at step {i+1}")
                return False
            current = next_state
        
        final_grid = self.tuple_to_grid(current)
        if not self.is_solved(final_grid):
            print("Final state is not solved")
            return False
        return True

    def get_grid_hash(self, grid):
        """Generate a hash for a grid to check uniqueness - includes exit position for extra safety"""
        grid_str = str(self.grid_to_tuple(grid))
        exit_str = str(self.exit_pos)
        combined_str = grid_str + exit_str
        return hashlib.sha256(combined_str.encode()).hexdigest()
    
    def generate_solution_moves(self, start_grid, solution_path):
        """Generate text description of moves"""
        moves = []
        current_grid = start_grid
        
        for i, next_state in enumerate(solution_path):
            next_grid = self.tuple_to_grid(next_state)
            moved_piece = None
            old_positions = []
            new_positions = []
            
            # Unique pieces in current grid
            current_pieces = set()
            for r in range(self.grid_size):
                for c in range(self.grid_size):
                    if current_grid[r][c] != '.':
                        current_pieces.add(current_grid[r][c])
            
            for piece in current_pieces:
                old_pos = self.find_piece_positions(current_grid, piece)
                new_pos = self.find_piece_positions(next_grid, piece)
                if old_pos != new_pos:
                    moved_piece = piece
                    old_positions = old_pos
                    new_positions = new_pos
                    break
            
            if moved_piece:
                # Convert to 1-indexed for display
                old_display = [(r+1, c+1) for r, c in old_positions]
                new_display = [(r+1, c+1) for r, c in new_positions]
                
                if len(old_positions) == 1:
                    moves.append(f"Step {i+1}: {moved_piece} [{old_display[0][0]},{old_display[0][1]}] -> [{new_display[0][0]},{new_display[0][1]}]")
                else:
                    moves.append(f"Step {i+1}: {moved_piece} {old_display} -> {new_display}")
            
            current_grid = next_grid
        
        return moves

    def draw_grid(self, grid, save_path=None, figsize=(8, 8)):
        """Draw the grid with pieces - no title, cleaner appearance"""
        fig, ax = plt.subplots(figsize=figsize)
        ax.set_xlim(0, self.grid_size)
        ax.set_ylim(0, self.grid_size)
        ax.set_xticks(range(self.grid_size + 1))
        ax.set_yticks(range(self.grid_size + 1))
        
        # Draw main grid lines
        for x in range(self.grid_size + 1):
            ax.plot([x, x], [0, self.grid_size], color='black', linewidth=2)
        for y in range(self.grid_size + 1):
            ax.plot([0, self.grid_size], [y, y], color='black', linewidth=2)
        
        ax.invert_yaxis()
        
        # Draw exit target
        er, ec = self.exit_pos
        target_rect = patches.Rectangle((ec, er), 1, 1,
                                        facecolor='none',
                                        edgecolor='red', linewidth=6)
        ax.add_patch(target_rect)
        
        # Color mapping
        colors = {
            'C': 'skyblue',
            'B1': 'lightcoral', 'B2': 'lightgreen', 'B3': 'plum', 
            'B4': 'khaki', 'B5': 'lightsalmon', 'B6': 'lightsteelblue',
            'H1': 'orange', 'H2': 'cyan', 'H3': 'yellow',
            'H4': 'pink', 'H5': 'lightgray', 'H6': 'lavender'
        }
        
        # Collect info
        piece_info = {}
        dotted_lines = []
        drawn_pieces = set()
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell not in drawn_pieces:
                    positions = self.find_piece_positions(grid, cell)
                    piece_info[cell] = positions
                    drawn_pieces.add(cell)
        
        # Draw pieces
        for piece_id, positions in piece_info.items():
            if len(positions) == 1:
                r, c = positions[0]
                ax.add_patch(
                    patches.Rectangle((c, r), 1, 1,
                                      facecolor=colors.get(piece_id, 'white'),
                                      edgecolor='black', linewidth=2)
                )
                ax.text(c + 0.5, r + 0.25, piece_id, ha='center', va='center',
                        fontsize=16, fontweight='bold')
            else:
                min_r = min(pos[0] for pos in positions)
                max_r = max(pos[0] for pos in positions)
                min_c = min(pos[1] for pos in positions)
                max_c = max(pos[1] for pos in positions)
                
                width = max_c - min_c + 1
                height = max_r - min_r + 1
                
                ax.add_patch(
                    patches.Rectangle((min_c, min_r), width, height,
                                      facecolor=colors.get(piece_id, 'white'),
                                      edgecolor='black', linewidth=2,
                                      zorder=3)
                )
                
                if width == 2:  # Horizontal 2x1
                    line_x = min_c + 1
                    line_y_start = min_r
                    line_y_end = min_r + 1
                    dotted_lines.append([(line_x, line_y_start), (line_x, line_y_end)])
                    label_x = line_x
                    label_y = min_r + 0.5
                else:  # Vertical 2x1
                    line_x_start = min_c
                    line_x_end = min_c + 1
                    line_y = min_r + 1
                    dotted_lines.append([(line_x_start, line_y), (line_x_end, line_y)])
                    label_x = min_c + 0.5
                    label_y = line_y
                
                ax.text(label_x, label_y, piece_id, ha='center', va='center',
                        fontsize=14, fontweight='bold', zorder=5, color='black')
        
        # Dotted lines
        for line in dotted_lines:
            start, end = line
            ax.plot([start[0], end[0]], [start[1], end[1]], 
                    color='black', linewidth=1, linestyle='--', alpha=0.7, zorder=4)
        
        # Coordinates in each cell
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                coord_text = f"({r+1},{c+1})"
                ax.text(c + 0.5, r + 0.8, coord_text, ha='center', va='center',
                        fontsize=12, color='black', fontweight='normal')
        
        # Target label
        ax.text(ec + 0.85, er + 0.5, 'TARGET', ha='right', va='center',
                fontsize=13, fontweight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', 
                          alpha=0.95, edgecolor='red', linewidth=2))
        
        ax.axis('off')
        
        if save_path:
            plt.savefig(save_path, bbox_inches='tight', dpi=150)
            plt.close()
        else:
            plt.tight_layout()
            plt.show()
    
    def generate_puzzle_json(self, grid, difficulty="unknown", num_1x1_blockers=0, num_2x1_blockers=0,
                           solution_length=0):
        """Generate JSON representation of puzzle state with embedded prompt"""
        # Find car position
        car_pos = self.find_piece_positions(grid, 'C')[0]
        
        # Convert positions to 1-indexed for JSON (matches visual display)
        car_pos_1indexed = [car_pos[0] + 1, car_pos[1] + 1]
        exit_pos_1indexed = [self.exit_pos[0] + 1, self.exit_pos[1] + 1]
        
        # Build pieces dictionary
        pieces = {}
        pieces['C'] = {"type": "car", "position": car_pos_1indexed}
        
        # Find all blockers
        processed_pieces = {'C'}
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell not in processed_pieces:
                    positions = self.find_piece_positions(grid, cell)
                    if len(positions) == 1:
                        pos_1indexed = [positions[0][0] + 1, positions[0][1] + 1]
                        pieces[cell] = {"type": "1x1_blocker", "position": pos_1indexed}
                    else:
                        pos_1indexed = [[pos[0] + 1, pos[1] + 1] for pos in positions]
                        if positions[0][0] == positions[1][0]:
                            piece_type = "2x1_horizontal_blocker"
                        else:
                            piece_type = "2x1_vertical_blocker"
                        pieces[cell] = {"type": piece_type, "positions": pos_1indexed}
                    processed_pieces.add(cell)
        
        # Generate the prompt for this specific puzzle (NO STRATEGY HINTS)
        prompt = self.generate_puzzle_specific_prompt_no_strategy(grid)
        
        # Create JSON structure
        puzzle_json = {
            "grid": grid,
            "car_position": car_pos_1indexed,
            "exit_position": exit_pos_1indexed,
            "pieces": pieces,
            "puzzle_info": {
                "difficulty": difficulty,
                "num_1x1_blockers": num_1x1_blockers,
                "num_2x1_blockers": num_2x1_blockers,
                "total_moves_in_solution": solution_length,
                "grid_size": f"{self.grid_size}x{self.grid_size}",
                "coordinate_system": "1-indexed, [row,col] format where [1,1] is top-left"
            },
            "prompt": prompt
        }
        
        return puzzle_json

    def generate_puzzle_specific_prompt_no_strategy(self, grid):
        """Generate a prompt WITHOUT strategy hints for true evaluation"""
        car_pos = self.find_piece_positions(grid, 'C')[0]
        car_pos_1indexed = (car_pos[0] + 1, car_pos[1] + 1)
        exit_pos_1indexed = (self.exit_pos[0] + 1, self.exit_pos[1] + 1)
        
        blockers_1x1 = []
        blockers_2x1 = []
        
        pieces = set()
        for r in range(self.grid_size):
            for c in range(self.grid_size):
                cell = grid[r][c]
                if cell != '.' and cell != 'C' and cell not in pieces:
                    pieces.add(cell)
                    positions = self.find_piece_positions(grid, cell)
                    if len(positions) == 1:
                        pos_1indexed = (positions[0][0] + 1, positions[0][1] + 1)
                        blockers_1x1.append(f"{cell} at [{pos_1indexed[0]},{pos_1indexed[1]}]")
                    else:
                        pos_1indexed = [(pos[0] + 1, pos[1] + 1) for pos in positions]
                        positions_str = ", ".join([f"[{p[0]},{p[1]}]" for p in pos_1indexed])
                        if positions[0][0] == positions[1][0]:
                            orientation = "horizontal"
                        else:
                            orientation = "vertical"
                        blockers_2x1.append(f"{cell} ({orientation}) at {positions_str}")

        prompt = f"""Task: You have been given a 4x4 Rush Hour puzzle above which you need to solve. Move car "C" from position [{car_pos_1indexed[0]},{car_pos_1indexed[1]}] to the TARGET at position [{exit_pos_1indexed[0]},{exit_pos_1indexed[1]}].

Current Pieces:
- Car "C": Position [{car_pos_1indexed[0]},{car_pos_1indexed[1]}]
- 1x1 Blockers: {', '.join(blockers_1x1) if blockers_1x1 else 'None'}
- 2x1 Blockers: {', '.join(blockers_2x1) if blockers_2x1 else 'None'}
- TARGET: Position [{exit_pos_1indexed[0]},{exit_pos_1indexed[1]}]

Rules:
- Any piece can move UP, DOWN, LEFT, or RIGHT by exactly one square
- For 2x1 blockers: The entire piece moves together as a unit
- Pieces cannot move outside the 4x4 grid
- Pieces cannot move into occupied squares
- No two pieces can occupy the same square
- Goal: Move car "C" to the TARGET position

Coordinate System: [row,col] format where [1,1] is top-left, [4,4] is bottom-right

Provide your solution as:
<solution>
Step 1: [PIECE] [start_position] -> [end_position]
Step 2: [PIECE] [start_position] -> [end_position]
...
</solution>

For 1x1 pieces (car "C" and blockers "B1", "B2", etc.):
- Use single coordinate: C [2,1] -> [2,2]

For 2x1 pieces (blockers "H1", "H2", etc.):
- List both coordinates: H1 [[1,1],[1,2]] -> [[2,1],[2,2]]

Example response format:
<solution>
Step 1: C [2,1] -> [2,2]
Step 2: H1 [[1,3],[1,4]] -> [[2,3],[2,4]]  
Step 3: B1 [3,2] -> [3,1]
Step 4: C [2,2] -> [2,4]
</solution>"""
        return prompt

    def generate_master_puzzle_configs(self, num_puzzles=150, random_seed=42):
        """
        Generate master list ensuring exactly 50 easy, 50 medium, 50 hard base puzzles.
        """
        print("🎯 Generating Master Puzzle Configuration List")
        print("=" * 60)
        
        # Set seed for reproducibility
        random.seed(random_seed)
        
        master_configs = []
        seen_hashes = set()
        
        # Track difficulty distribution
        difficulty_counts = {"easy": 0, "medium": 0, "hard": 0}
        base_puzzle_counts = {"easy": 0, "medium": 0, "hard": 0}
        
        base_puzzle_num = 0
        attempts_total = 0
        
        MAX_ATTEMPTS_PER_PUZZLE = 100
        
        while base_puzzle_num < num_puzzles and attempts_total < num_puzzles * MAX_ATTEMPTS_PER_PUZZLE:
            attempts_total += 1
            
            try:
                base_puzzle_num += 1
                num_1x1, num_2x1, difficulty = self.get_difficulty_config_for_base(base_puzzle_num)
                
                base_puzzle = self.generate_puzzle(num_1x1_blockers=num_1x1, num_2x1_blockers=num_2x1)
                base_puzzle_counts[difficulty] += 1
                
                # Check solvability and uniqueness (no transformations)
                solution = self.bfs_solve(base_puzzle)
                if solution is None:
                    base_puzzle_num -= 1
                    continue
                
                # Check uniqueness
                grid_hash = self.get_grid_hash(base_puzzle)
                if grid_hash in seen_hashes:
                    base_puzzle_num -= 1
                    continue
                
                # Accept this configuration
                seen_hashes.add(grid_hash)
                difficulty_counts[difficulty] += 1
                
                # Create configuration record
                config = {
                    'config_id': len(master_configs) + 1,
                    'base_puzzle_num': base_puzzle_num,
                    'grid': base_puzzle,
                    'difficulty': difficulty,
                    'num_1x1_blockers': num_1x1,
                    'num_2x1_blockers': num_2x1,
                    'grid_hash': grid_hash,
                    'solution_length': len(solution),
                    'solution': solution
                }
                
                master_configs.append(config)
                
                if len(master_configs) % 25 == 0:
                    print(f"✓ Generated {len(master_configs)} configurations...")
                    print(f"   Base puzzles: Easy={base_puzzle_counts['easy']}, Medium={base_puzzle_counts['medium']}, Hard={base_puzzle_counts['hard']}")
                    print(f"   Total configs: Easy={difficulty_counts['easy']}, Medium={difficulty_counts['medium']}, Hard={difficulty_counts['hard']}")
            
            except Exception as e:
                base_puzzle_num -= 1  # Retry this puzzle number
                continue
        
        print(f"\n🎉 Master configuration generation complete!")
        print(f"Generated {len(master_configs)} unique configurations")
        print(f"Base puzzles: Easy={base_puzzle_counts['easy']}, Medium={base_puzzle_counts['medium']}, Hard={base_puzzle_counts['hard']}")
        print(f"Total configs: Easy={difficulty_counts['easy']}, Medium={difficulty_counts['medium']}, Hard={difficulty_counts['hard']}")
        print(f"Unique hashes verified: {len(seen_hashes)}")
        
        # Save master configuration list
        master_file = "data/4x4/master_puzzle_configs.json"
        os.makedirs("data/4x4", exist_ok=True)
        
        # Convert to JSON-serializable format
        json_configs = []
        for config in master_configs:
            json_config = config.copy()
            json_config['solution'] = [list(sol) for sol in config['solution']]
            json_configs.append(json_config)
        
        with open(master_file, 'w') as f:
            json.dump(json_configs, f, indent=2)
        
        # Also save as pickle
        with open("data/4x4/master_puzzle_configs.pkl", 'wb') as f:
            pickle.dump(master_configs, f)
        
        print(f"Master configuration saved to: {master_file}")
        
        # Generate CSV documentation
        self.generate_csv_documentation(master_configs, "data/4x4/puzzle_catalog.csv")
        
        return master_configs

    def generate_csv_documentation(self, master_configs, csv_path):
        """Generate CSV file documenting all puzzles with their difficulty labels"""
        with open(csv_path, 'w', newline='') as csvfile:
            fieldnames = ['puzzle_id', 'folder_name', 'base_puzzle', 
                         'difficulty', 'num_1x1_blockers', 'num_2x1_blockers', 
                         'solution_length', 'exit_position']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()
            for config in master_configs:
                base_num = config['base_puzzle_num']
                folder_name = f"puzzle{base_num}"
                
                writer.writerow({
                    'puzzle_id': config['config_id'],
                    'folder_name': folder_name,
                    'base_puzzle': base_num,
                    'difficulty': config['difficulty'],
                    'num_1x1_blockers': config['num_1x1_blockers'],
                    'num_2x1_blockers': config['num_2x1_blockers'],
                    'solution_length': config['solution_length'],
                    'exit_position': f"[2,4]"  # Fixed exit position for 4x4
                })
        
        print(f"CSV documentation saved to: {csv_path}")
        
        # Print summary statistics
        difficulty_stats = {"easy": 0, "medium": 0, "hard": 0}
        for config in master_configs:
            difficulty_stats[config['difficulty']] += 1
        
        print("\nDifficulty Distribution in CSV:")
        print(f"  Easy: {difficulty_stats['easy']} puzzles")
        print(f"  Medium: {difficulty_stats['medium']} puzzles")
        print(f"  Hard: {difficulty_stats['hard']} puzzles")
        print(f"  Total: {sum(difficulty_stats.values())} puzzles")

    def create_dataset_from_master_configs(self, master_configs, base_folder="data/4x4"):
        """Create dataset from pre-generated master configuration list."""
        print("\n🎯 Creating Dataset from Master Configurations")
        print("=" * 60)
        
        if not os.path.exists(base_folder):
            os.makedirs(base_folder)
        
        processed_count = 0
        
        for config in master_configs:
            try:
                # Extract configuration data
                config_id = config['config_id']
                base_num = config['base_puzzle_num']
                grid = config['grid']
                difficulty = config['difficulty']
                num_1x1_blockers = config['num_1x1_blockers']
                num_2x1_blockers = config['num_2x1_blockers']
                solution = config['solution']
                
                # Create folder name (no transformations)
                folder_name = f"puzzle{base_num}"
                puzzle_folder = os.path.join(base_folder, folder_name)
                if not os.path.exists(puzzle_folder):
                    os.makedirs(puzzle_folder)
                
                # Generate and save puzzle-specific prompt (NO STRATEGY)
                puzzle_prompt = self.generate_puzzle_specific_prompt_no_strategy(grid)
                with open(os.path.join(puzzle_folder, "prompt.txt"), 'w') as f:
                    f.write(puzzle_prompt)
                
                # Generate and save JSON representation
                puzzle_json = self.generate_puzzle_json(
                    grid=grid,
                    difficulty=difficulty,
                    num_1x1_blockers=num_1x1_blockers,
                    num_2x1_blockers=num_2x1_blockers,
                    solution_length=len(solution)
                )
                with open(os.path.join(puzzle_folder, "puzzle_state.json"), 'w') as f:
                    json.dump(puzzle_json, f, indent=2)
                
                # Save initial state image
                self.draw_grid(grid, save_path=os.path.join(puzzle_folder, "initial_state.png"))
                
                # Save final solved state image
                if solution:
                    final_state = self.tuple_to_grid(solution[-1])
                    self.draw_grid(final_state, save_path=os.path.join(puzzle_folder, "final_solved_state.png"))
                
                # Generate solution text
                moves = self.generate_solution_moves(grid, solution)
                
                solution_file = os.path.join(puzzle_folder, "solution.txt")
                with open(solution_file, 'w') as f:
                    f.write(f"Puzzle: {folder_name}\n")
                    f.write(f"Configuration ID: {config_id}\n")
                    f.write(f"Total moves: {len(solution)}\n")
                    f.write(f"Exit position: [2,4]\n")
                    f.write(f"Difficulty: {difficulty} ({num_1x1_blockers} 1x1 blockers, {num_2x1_blockers} 2x1 blockers)\n")
                    f.write("Coordinate system: [row,col] where [1,1] is top-left\n\n")
                    
                    f.write("SOLUTION:\n")
                    f.write("=" * 40 + "\n")
                    for move in moves:
                        f.write(move + "\n")
                
                processed_count += 1
                
                if processed_count % 25 == 0:
                    print(f"✓ Processed {processed_count}/{len(master_configs)} configurations...")
            
            except Exception as e:
                print(f"Error processing configuration {config.get('config_id', 'unknown')}: {e}")
                continue
        
        print(f"\n🎉 Dataset creation complete!")
        print(f"Processed {processed_count}/{len(master_configs)} configurations")
        print(f"Dataset saved in '{base_folder}' folder")
        
        return processed_count

    def load_master_configs(self, filepath="data/4x4/master_puzzle_configs.pkl"):
        """Load master configuration list from file"""
        try:
            with open(filepath, 'rb') as f:
                return pickle.load(f)
        except FileNotFoundError:
            print(f"Master config file not found: {filepath}")
            return None

    def create_full_dataset(self, num_puzzles=150, random_seed=42):
        """
        Complete workflow: Generate master configs then create dataset.
        Ensures exactly 50 easy, 50 medium, 50 hard base puzzles.
        """
        print("🚀 Starting Full 4x4 Rush Hour Dataset Generation")
        print("=" * 60)
        print(f"Target: {num_puzzles} base puzzles (no transformations)")
        print(f"Distribution: 50 easy + 50 medium + 50 hard = 150 base puzzles")
        print(f"Random seed: {random_seed} (for reproducibility)")
        print()
        
        # CELL 1: Generate master configuration list
        master_configs = self.generate_master_puzzle_configs(num_puzzles, random_seed)
        
        # CELL 2: Create dataset from master configurations
        processed_count = self.create_dataset_from_master_configs(master_configs)
        
        print(f"\n✅ COMPLETE: Generated {len(master_configs)} unique puzzle configurations")
        print(f"✅ COMPLETE: Processed {processed_count} puzzle files")
        print("\nDataset features:")
        print("- ✅ Reproducible generation using fixed random seed")
        print(f"- ✅ All {len(master_configs)} configurations guaranteed unique")
        print("- ✅ Single optimal solution per puzzle")
        print("- ✅ No strategy hints in prompts (true evaluation)")
        print("- ✅ CSV catalog with difficulty labels")
        print("- ✅ Exactly 50 easy, 50 medium, 50 hard puzzles")
        print("- ✅ No transformations - only base puzzles")

    def test_solver(self):
        """Test the BFS solver with known cases"""
        print("Testing 4x4 BFS Solver...")
        
        # Test 1: Generate and solve random puzzle
        random.seed(42)
        test_grid = self.generate_puzzle(num_1x1_blockers=3, num_2x1_blockers=1)
        solution = self.bfs_solve(test_grid)
        assert solution is not None, "Test 1 failed: No solution found"
        assert self.verify_solution(test_grid, solution), "Test 1 verification failed"
        print(f"✓ Test 1 passed: Random puzzle solved in {len(solution)} moves")
        
        # Test 2: Generate another puzzle
        random.seed(123)
        test_grid = self.generate_puzzle(num_1x1_blockers=2, num_2x1_blockers=2)
        solution = self.bfs_solve(test_grid)
        assert solution is not None, "Test 2 failed: No solution found"
        assert self.verify_solution(test_grid, solution), "Test 2 verification failed"
        print(f"✓ Test 2 passed: Puzzle solved in {len(solution)} moves")
        
        print("All 4x4 BFS solver tests passed! ✅\n")

# ----- Main Execution -----

if __name__ == "__main__":
    print("Creating 4x4 Rush Hour Dataset with Fixed Difficulty Distribution")
    print("This ensures exactly 50 easy, 50 medium, 50 hard base puzzles.")
    print("NO TRANSFORMATIONS - Only base puzzles will be generated.")
    print()
    
    game = RushHour4x4()
    
    # Verify BFS solver first
    game.test_solver()
    
    # Generate fresh dataset with proper distribution (no transformations)
    game.create_full_dataset(num_puzzles=150, random_seed=42)
    
    # Option 2: Load existing master configs and regenerate dataset
    # master_configs = game.load_master_configs()
    # if master_configs:
    #     game.create_dataset_from_master_configs(master_configs)
    # else:
    #     print("No existing master configs found. Run Option 1 first.")

Creating 4x4 Rush Hour Dataset with Fixed Difficulty Distribution
This ensures exactly 50 easy, 50 medium, 50 hard base puzzles.
NO TRANSFORMATIONS - Only base puzzles will be generated.

Testing 4x4 BFS Solver...
✓ Test 1 passed: Random puzzle solved in 1 moves
✓ Test 2 passed: Puzzle solved in 6 moves
All 4x4 BFS solver tests passed! ✅

🚀 Starting Full 4x4 Rush Hour Dataset Generation
Target: 150 base puzzles (no transformations)
Distribution: 50 easy + 50 medium + 50 hard = 150 base puzzles
Random seed: 42 (for reproducibility)

🎯 Generating Master Puzzle Configuration List
✓ Generated 25 configurations...
   Base puzzles: Easy=25, Medium=0, Hard=0
   Total configs: Easy=25, Medium=0, Hard=0
✓ Generated 50 configurations...
   Base puzzles: Easy=50, Medium=0, Hard=0
   Total configs: Easy=50, Medium=0, Hard=0
✓ Generated 75 configurations...
   Base puzzles: Easy=50, Medium=25, Hard=0
   Total configs: Easy=50, Medium=25, Hard=0
✓ Generated 100 configurations...
   Base puzzles: Eas